In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

In [ ]:
from transformers import BitsAndBytesConfig

import torch

from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

hf_pipe = HuggingFacePipeline.from_model_id(
    model_id=model_name,
    task="text-generation",
    device_map="auto",
    device=None,
    pipeline_kwargs={
        "max_new_tokens": 1024,
    },
    model_kwargs={
        "torch_dtype": "auto",
        "quantization_config": bnb_config,
        "attn_implementation": "flash_attention_2",
        "low_cpu_mem_usage": True
    }
)

In [ ]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)
chain = prompt | hf_pipe
question = "What is electroencephalography?"

for chunk in chain.stream("Write me a song about sparkling water."):
    print(chunk, end="", flush=True)

# print(chain.invoke({"question": question}))

In [ ]:
from transformers import AutoTokenizer
from transformers import TextStreamer
from transformers import pipeline

import torch

tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer)

pipe = pipeline(
    "text-generation",
    model=model_name,
    model_kwargs=
    {
        "load_in_4bit": True,
        "bnb_4bit_compute_dtype": torch.float16
    },
    device_map="auto",
    tokenizer=tokenizer,
    streamer=streamer
)

In [ ]:
print(f'Model size : {pipe.model.get_memory_footprint() // 1024 ** 2} MB')
pipe.model.is_loaded_in_4bit

In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline(pipeline=pipe)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "electroencephalography is",
    }
]

_ = hf.invoke(messages)

In [ ]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)
chain = prompt | hf
question = "What is electroencephalography?"

_ = hf.invoke(question)